# 🔹 4. 数据处理模块

### 1、torch.utils.data.Dataset：定义自定义数据集

### 2、torch.utils.data.DataLoader：批量加载数据

### 3、torchvision.datasets：常用数据集（MNIST、CIFAR、ImageNet 等）

### 4、torchvision.transforms：图像预处理（裁剪、缩放、归一化、增强等）

# 数据加载方式

## 1. 🔹 最核心和基础的方式：`Dataset` + `DataLoader`
这是 `PyTorch` 数据加载的基石，几乎所有其他方式都是在此基础上构建的。它采用了“`组合`”的设计模式，将数据来源 (`Dataset`) 和数据读取策略 (`DataLoader`) 分离。

### 🔹 一、自定义 `Dataset` (最常用)
这是最灵活的方式，你需要继承 `torch.utils.data.Dataset` 类并实现三个方法：

- `__init__`: 初始化，用于读取文件路径、标签等元信息。

- `__len__`: 返回数据集的总大小。

- `__getitem__`: 根据索引 `index` 返回一个数据样本（如图像张量和标签）。

示例（加载图像分类数据集）：

In [ ]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os

class CustomImageDataset(Dataset):
    """
    自定义图像数据集类，继承自PyTorch的Dataset类
    用于加载指定目录下的图像文件并支持数据转换
    
    参数:
        img_dir (str): 包含图像文件的目录路径
        transform (callable, optional): 应用于图像的转换/增强操作
    """
    
    def __init__(self, img_dir, transform=None):
        """
        初始化数据集
        
        参数:
            img_dir: 图像目录路径
            transform: 图像预处理/增强的转换函数
        """
        self.img_dir = img_dir               # 存储图像目录路径
        self.img_names = os.listdir(img_dir) # 获取目录下所有文件名(假设都是图片)
        self.transform = transform           # 存储转换函数
    
    def __len__(self):
        """返回数据集中的样本数量"""
        return len(self.img_names)
    
    def __getitem__(self, idx):
        """
        获取单个样本(图像和标签)
        
        参数:
            idx: 样本索引
            
        返回:
            元组(image, label): 处理后的图像和对应的标签
        """
        img_path = os.path.join(self.img_dir, self.img_names[idx])  # 构建完整图像路径
        image = Image.open(img_path)  # 使用PIL打开图像文件
        
        # 这里需要根据实际情况实现标签获取逻辑
        # 例如: 从文件名解析、从单独标签文件读取等
        label = ... # 根据图片名或其他方式获取标签
        
        if self.transform:
            image = self.transform(image)  # 应用转换(如归一化、裁剪等)
            
        return image, label  # 返回图像和标签对


# 使用示例
# 创建数据集实例
dataset = CustomImageDataset(
    'path/to/images',       # 图像目录路径
    transform=my_transform  # 图像转换操作(如torchvision.transforms中的组合)                         
)

# 创建数据加载器
dataloader = DataLoader(
    dataset,        # 数据集对象
    batch_size=32,  # 每批加载的样本数
    shuffle=True,   # 是否打乱数据顺序
    num_workers=4   # 使用4个子进程加载数据
)

### 🔹二、内置数据集加载

`PyTorch` 在 `torchvision.datasets`、`torchtext.datasets`、`torchaudio.datasets` 中提供了许多常见的公共数据集。
- 特点：直接调用，支持 `transforms`，一般配合 `DataLoader` 使用。

示例：

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.ToTensor()
train_data = datasets.MNIST(root="./data", 
                            train=True,  # 测试集 写 False
                            transform=transform, # 数据转换
                            download=True)
train_loader = DataLoader(train_data,    # 要加载的数据集
                          batch_size=64, # 批量大小
                          shuffle=True,  # 是否打乱顺序，验证、测试不需要 
                          pin_memory= True, # 是否将数据锁页内存中，加速 GPU 数据传输（在 GPU 训练时通常设为 True）
                          num_workers=2)  # 用2个子进程来加载数据，加速数据读取的关键

### 🔹 三、ImageFolder & DatasetFolder

`ImageFolder`：针对按类别存放的图片数据集，文件夹名即标签。

`DatasetFolder`：更通用，支持自定义文件格式（比如 .npy、.wav）。

示例：

In [ ]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import transforms

transform = transforms.ToTensor()
dataset = ImageFolder(root="./data/train", transform=transform)
loader = DataLoader(dataset, batch_size=32, shuffle=True)


### 🔹 四、`TensorDataset` & `ConcatDataset` & `Subset`

- `TensorDataset`：直接把 `Tensor` 打包成数据集。

- `ConcatDataset`：多个数据集合并。

- `Subset`：从一个数据集中抽取部分样本。

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
import torch

x = torch.randn(100, 3, 28, 28)
y = torch.randint(0, 10, (100,))
dataset = TensorDataset(x, y)
loader = DataLoader(dataset, batch_size=16)


### 🔹 五、DataLoader 参数方式

`DataLoader` 提供了多种灵活的加载策略：

- `batch_size` 批大小

- `shuffle` 是否打乱

- `num_workers` 多进程并行加载

- `collate_fn` 自定义 `batch` 组装方式（比如处理不同长度序列）

- `pin_memory`、`persistent_workers` 等性能优化参数

---

### 🔹 六、Streaming / IterableDataset

如果数据量特别大，不能一次性存到硬盘或内存，可以用 `IterableDataset` 来流式读取，比如从：
- 数据库
- 日志流
- 在线数据生成器

示例：

In [ ]:
from torch.utils.data import IterableDataset

class MyIterableDataset(IterableDataset):
    def __iter__(self):
        for i in range(1000):
            yield i

dataset = MyIterableDataset()
for data in dataset:
    print(data)


### 🔹 七、其他常见方式

1.直接加载到 Tensor（小数据集常用，直接 torch.tensor(data)）。

2.HDF5 / LMDB / Parquet 等格式，通常配合 Dataset 自定义读取。

3.WebDataset（tar 分片存储，常用于大规模分布式训练）。

4.第三方库（Hugging Face Datasets、TorchData、Petastorm 等）。

## 2. 基于索引与可迭代的风格：`MapStyleDataset` vs `IterableDataset`
上述的自定义 `Dataset` 属于 `Map-style Dataset`。`PyTorch` 还提供了另一种风格：

a)` Map-style Dataset` (主流):\
实现了 `__getitem__` 和 `__len__` 方法。它假设数据集是一个映射（`Map`），可以通过索引（如 0, 1, 2...）随机访问任何样本。这对于存储在磁盘或内存中的标准数据集非常有效。

b) `Iterable-style Dataset`:\
需要继承 `torch.utils.data.IterableDataset` 并实现 `__iter__` 方法。它返回一个数据流的迭代器。适用于：

- 数据流：数据是实时生成的（如传感器数据）或来自网络流。

- 无法随机访问：数据存储在无法简单索引的格式中（如巨大的二进制文件，数据库查询结果）。

- 避免随机读取：顺序读取比随机读取快得多的时候（如读取磁带）。

示例（读取一个巨大的二进制文件流）：

In [ ]:
from torch.utils.data import IterableDataset, DataLoader

class BinaryIterableDataset(IterableDataset):
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        with open(self.filename, ‘rb’) as f:
            while True:
                data = f.read(1024) # 每次读取1024字节
                if not data:
                    break
                # 将数据转换为张量
                yield torch.frombuffer(data, dtype=torch.float32)

## 3. 高阶和扩展库
对于更复杂或大规模的场景，社区和官方也提供了更强大的工具。

a) `DataPipe` (PyTorch 1.10+ 官方新特性)\
这是 `PyTorch` 官方推荐的用于替代之前 `Dataset` 的新 `API`，旨在提供更模块化、可组合、可扩展的数据处理流程。它与 `DataLoader` 兼容。

- 核心思想：将数据加载和预处理步骤分解成多个小的、可重用的 `DataPipe`，然后像搭积木一样将它们连接起来。

示例：

In [ ]:
from torchdata.datapipes.iter import IterableWrapper, FileOpener

# 创建一个简单的数据处理流程
datapipe = IterableWrapper([‘data_file_1.txt’, ‘data_file_2.txt’])
datapipe = FileOpener(datapipe, mode=‘r’)
datapipe = datapipe.parse_csv(delimiter=‘,’)

dataloader = DataLoader(datapipe, batch_size=32)

b) `torchdistx` 和 `DataLoader2` (实验性) \
为了应对更极端的分布式和数据加载需求，`PyTorch` 正在开发下一代数据加载器 (`DataLoader2`) 和 `torchdistx `库，用于在数据加载时进行延迟张量初始化，可以极大减少内存占用。

c) 第三方库 
- `WebDataset`: 非常流行用于超大规模数据集，它将数据集存储为 `tar` 文件，每个样本是 `tar` 内的一个文件。可以像处理普通`Dataset` 一样高效地流式读取，非常适合云存储。

- `NVIDIA DALI `(Data Loading Library): 一个用于数据加载和预处理的 `GPU` 加速库。它将整个数据预处理管道（解码、裁剪、归一化等）放到 `GPU` 上执行，极大减少了 CPU 的瓶颈。

- `PyTorch Geometric` (PyG) / `Deep Graph Library` (DGL): 图神经网络库，它们提供了自己专用的 `DataLoader `来处理图数据，可以处理可变大小的图并组成批处理。

---
### 总结与选择指南
| 方式                         | 适用场景                               | 优点                         | 缺点                             |
|------------------------------|----------------------------------------|------------------------------|----------------------------------|
| 自定义 Dataset + DataLoader  | 绝大多数情况，中小规模数据集           | 极度灵活，完全控制           | 需要自己写代码                   |
| 内置 Dataset                 | 快速开始，基准测试                     | 极简，开箱即用               | 只适用于特定数据集               |
| IterableDataset              | 数据流、数据库、顺序读取优势大         | 节省内存，处理无限数据流     | 无法随机打乱（或很难），无法直接获取长度 |
| DataPipe                     | 构建复杂、可复用的数据管道             | 模块化，官方未来方向         | 相对较新，生态还在发展中         |
| WebDataset                   | 超大规模数据集（TB/PB级），云存储      | 高效流式读取，格式简单       | 需要将数据打包成 tar 格式        |
| NVIDIA DALI                  | 数据预处理是性能瓶颈时                 | GPU 加速预处理，性能极高     | 增加系统复杂性，需要学习新 API   |


---

# torchvision.datasets.ImageFolder
这是 `PyTorch` 里最常见的 图像分类数据集`读取方式`，尤其适合存放在 `文件夹` 里的图片数据。

## 1. 基本作用

`ImageFolder` 假设你的数据按照下面的目录结构存放：

In [ ]:
root/          # 根目录
  ├── class1/  # 类别1
  │     ├── img001.png
  │     ├── img002.png
  │     └── ...
  ├── class2/  # 类别2
  │     ├── img003.png
  │     ├── img004.png
  │     └── ...
  └── ...


- 每个子文件夹的名字（`class1`, `class2`）就是类别标签。

- `ImageFolder` 会自动给类别分配 索引 (`int`)，比如：

In [ ]:
dataset.classes   # ['class1', 'class2']
dataset.class_to_idx   # {'class1': 0, 'class2': 1}


## 2. 主要参数讲解

In [ ]:
torchvision.datasets.ImageFolder(
    root: str,
    transform: Optional[Callable] = None,
    target_transform: Optional[Callable] = None,
    loader: Callable[[str], Any] = default_loader,
    is_valid_file: Optional[Callable[[str], bool]] = None
)


| 参数                     | 作用                                                     |
| ---------------------- | ------------------------------------------------------ |
| **`root`**             | 数据集的根目录（必须），子文件夹的名字就是类别。                               |
| **`transform`**        | 作用在 **图片** 上的预处理操作（如 `Resize`、`ToTensor`、`Normalize`）。 |
| **`target_transform`** | 作用在 **标签** 上的预处理（比如标签编码或 one-hot 转换）。                  |
| **`loader`**           | 指定如何读取图片，默认是用 `PIL.Image.open` 打开。                     |
| **`is_valid_file`**    | 过滤器函数，返回 `True/False`，决定文件是否有效。例如：只保留 `.jpg` 文件。       |


---
## 3. 使用示例
### （1）最基础的用法

In [ ]:
from torchvision import datasets, transforms

# 定义图像预处理
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 调整大小
    transforms.ToTensor(),          # 转换为 Tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                         std=[0.5, 0.5, 0.5])  # 标准化
])

# 加载数据集
dataset = datasets.ImageFolder(root="./data/train", transform=transform)

print(dataset.classes)        # 类别名字
print(dataset.class_to_idx)   # 类别 -> 索引映射

img, label = dataset[0]       # 取第一张图片和对应标签
print(img.shape, label)

### （2）结合 DataLoader

In [ ]:
from torch.utils.data import DataLoader

dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

for images, labels in dataloader:
    print(images.shape, labels.shape)
    break


### （3）只读取特定格式的文件

In [ ]:
dataset = datasets.ImageFolder(
    root="./data/train",
    transform=transform,
    is_valid_file=lambda path: path.endswith(".jpg")  # 只保留 jpg 文件
)


### （4）自定义标签处理

比如：把数字标签转成 `one-hot`：

In [ ]:
import torch

target_transform = lambda y: torch.nn.functional.one_hot(torch.tensor(y), num_classes=2)

dataset = datasets.ImageFolder(
    root="./data/train",
    transform=transform,
    target_transform=target_transform
)

img, label = dataset[0]
print(label)  # one-hot 向量


## ✅ 总结：

- `ImageFolder` 适合目录结构整齐的数据集。

- 常用参数是 `root`、`transform`、`target_transform`。

- 搭配 `DataLoader` 就能批量加载训练数据。